In [11]:
import sys
import numpy as np
from os import listdir
from os.path import isfile, join
import pandas as pd

from datetime import date
from datetime import datetime
import warnings

warnings.filterwarnings(action='ignore')
pd.options.display.max_columns = None

# Data Overview & Import

## REGISTERED DATA

In [12]:
#set path

#registered_path = r'C:\Users\samanthaca\canadacharities\data\raw\Registered'
registered_path = r'E:\Projects\canadacharities\data\raw\Registered'

In [13]:
#read in file names
registered_files = [f for f in listdir(registered_path) if isfile(join(registered_path, f))]

#create dic of dfs
dfs = {file.split('.csv')[0] : pd.read_csv(registered_path + '\\' + file, encoding='latin-1', dtype=str) for file in registered_files}

In [14]:
#preview df names
for key in dfs.keys(): print(key)

# Category_Sub-Category
# Country
# Designation
# Form Versioning Details
# Programs
# Province
# US State
Financial Section A_ B and C
Financial Section D & Schedule 6
Gift
Ident
New and Ongoing Programs
Schedule 1_ Foundations
Schedule 2_ Activities Outside Canada
Schedule 3_ Compensation
Schedule 5_ Non-Cash gifts
Schedule 7_ Description
Schedule 7_ Political Activities - Outside Canada
Schedule 7_ Political Activities - Resources
Trustee


In [15]:
#setting up lookup tables
category_table = dfs['# Category_Sub-Category']
designation_table = dfs['# Designation']
programs_table = dfs['# Programs']

category_table = category_table[['Category Code','Category English Desc', 'Sub-Category Code', 'Sub-Category English Desc']] \
                .rename(columns={'Category English Desc':'Category', 'Sub-Category English Desc':'Sub-Category'})
    
designation_table = designation_table[['Designation Code', 'Description_E']] \
                    .rename(columns={'Description_E':'Designation'})
    
programs_table = programs_table[['Line 1200, 1210, 1220', 'English Description']] \
                .rename(columns={'Line 1200, 1210, 1220':'Program Code', 'English Description':'Program'})

# REVOKED/ANNULLED DATA

In [16]:
#set path
revoked_path = r'E:\projects\canadacharities\data\raw\Revoked and Annulled'
#revoked_path = r'C:\Users\samanthaca\canadacharities\data\raw\Revoked and Annulled'

revoked_files = [f for f in listdir(revoked_path) if isfile(join(revoked_path, f))]

#create dic of dfs
revoked_dfs = {file.split('.csv')[0] : pd.read_csv(revoked_path + '\\' + file, encoding='latin-1', dtype=str) for file in revoked_files}

In [17]:
#preview dfs
for key in revoked_dfs.keys(): print(key)

# Category_Sub-Category
# Country
# Designation
# Province
# Reason
# US State
List of registered charities
List of revoked & annulled charities


In [18]:
#read in revoked reason codes
reason_table = revoked_dfs['# Reason']
reason_table = reason_table[['ReasonCode', 'DescriptionE']].rename(columns={'DescriptionE': 'Revoked Reason'})

#join tables together
revoked_df = pd.merge(revoked_dfs['List of revoked & annulled charities'], 
                      reason_table, left_on='Reason code', right_on='ReasonCode', how='left')

#clean up data
revoked_df['Revoked Date'] = revoked_df['Effective date of status'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))

#filter for revoked charities whose status was revoked since 2018, the year for which we have charity data
revoked_df = revoked_df[revoked_df['Revoked Date'] >= pd.Timestamp(2018,1,1)]
revoked_df = revoked_df[['BN/Registration number', 'Legal name', 'Revoked Date', 'Revoked Reason']]

# Data setup

Let's create a new dataset to characterize each charity, using the lookup tables to fill in coded information.

In [19]:
#merge charity identification data with financial data
df = pd.merge(dfs['Ident'], dfs['Financial Section A_ B and C'], left_on='BN/Registration Number', right_on='BN/Registration number', how='outer')
df = pd.merge(df, dfs['Financial Section D & Schedule 6'], left_on='BN/Registration Number', right_on='BN/Registration number', how='outer')


In [20]:
#prepare columns for merging
df['Category code'] = df['Category code'].astype(int).astype(str)
df['Sub-category code'] = df['Sub-category code'].astype(int).astype(str)
df.columns = df.columns.str.title()
df = df.rename(columns={'Bn/Registration Number':'BN/Registration Number'})

revoked_df.columns = revoked_df.columns.str.title()
revoked_df = revoked_df.rename(columns={'Bn/Registration Number':'BN/Registration Number'})

In [21]:
#merge new df with lookup tables
df = pd.merge(df, category_table, how='left', on='Category Code')

df = pd.merge(df, designation_table, how='left', on='Designation Code')

df = pd.merge(df, programs_table, how='left',
              left_on='1200 Program Area Code', 
              right_on='Program Code') \
              .rename(columns={'Program':'1200 Program Area'})

df = pd.merge(df, programs_table, how='left',
              left_on='1210 Program Area Code', 
              right_on='Program Code') \
              .rename(columns={'Program':'1210 Program Area'})

df = pd.merge(df, programs_table, how='left',
              left_on='1220 Program Area Code', 
              right_on='Program Code') \
              .rename(columns={'Program':'1220 Program Area'})

df = pd.merge(df, revoked_df, how='left', on='BN/Registration Number')

## Missing data

In [43]:
import locale
locale.setlocale(locale.LC_ALL, 'en_CA')
locale.format("%d", 1255000, grouping=True)

'1,255,000'

In [42]:
#checking for missing revoked data
filed_charities = len(df['BN/Registration Number'])
revoked_charities = len(revoked_df['BN/Registration Number'])
filed_revoked_charities = len(set(revoked_df['BN/Registration Number']) & set(df['BN/Registration Number']))

print("There are a total of {} charities with 2018 information filed. \nOf the total {} revoked charities since 2018, {}% have filed 2018 information. \nThis means that only {}% of total charities with filed information are revoked.".format(filed_charities, revoked_charities, round(filed_revoked_charities / revoked_charities * 100), round(filed_revoked_charities / filed_charities,4)))

There are a total of 1157144 charities with 2018 information filed. 
Of the total 4373 revoked charities since 2018, 33% have filed 2018 information. 
This means that only 0.0013% of total charities with filed information are revoked.


# Data cleaning

In [ ]:
#remove duplicate columns as a result of joins
df = df[df.columns.drop(list(df.filter(regex='_y')))]
df.columns = df.columns.str.rstrip('_x')

#keep only first row if column appears more than once
df.drop_duplicates(subset='BN/Registration Number', keep = 'first', inplace = True)

#create status flag
df['Status'] = np.nan
df.loc[df['Revoked Reason'].isnull(), 'Status'] = 'Registered'
df.loc[df['Status'] != 'Registered', 'Status'] = 'Revoked'

#Bring in data dictionary with data types to assign each column
data_dic = pd.read_csv(r'E:\Projects\canadacharities\data\external\data_dictionary.csv')

#make list of columns that need to be float type
float_list = list(data_dic.loc[data_dic['data_type']=='float','column_name'])

#make list of columns that need to be bool type
bool_list = list(data_dic.loc[data_dic['data_type']=='bool','column_name'])

#change columns in float list to float
df[float_list] = df[float_list].apply(lambda x: x.str.replace(',', '')) #remove leading $ character
df[float_list] = df[float_list].apply(lambda x: x.str.replace('$', '')) #remove commas from numbers
df[float_list] = df[float_list].apply(pd.to_numeric)

#change columns in bool list to bool
df[bool_list] = df[bool_list].apply(lambda x : np.where(x == 'Y', '1', np.where(x == 'N', '0', np.nan)))

#filtering for charities only (75% of total rows)
df = df[df['Designation']=='Charitable Organization']

'''
#removing columns with more than 50% of data missing, keeping 'Revoked Date'
columns = np.append(df.columns[df.isnull().mean() <= 0.5], ['Revoked Date', 'Revoked Reason'])
df = df[columns]
'''

#putting registration date into datetime format
df['Registration date'] = df['Registration date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))

#Montreal appears as both MONTREAL and MONTRÉAL - let's fix this
df.loc[df['City']=='MONTRÉAL', 'City'] = 'MONTREAL'

## Financial data

In [ ]:
#create metrics for comparing charities
df['assets_vs_liabilities'] = df['4200'] / df['4350']
df['revenue_vs_expenditures'] = df['4700'] / df['5100']
df['accumulated_vs_disbursed'] = df['5500'] / df['5510']

In [ ]:
#total revenue name [4700]
#total expenditure column name [5100]

revenue_list = list(data_dic.loc[data_dic['category']=='Revenue', 'column_name'])
expenditures_list = list(data_dic.loc[data_dic['category']=='Expenditures', 'column_name'])

#add suffix to revenue list to prevent saving over values
revenue_perc_list = ['{}_perc_revenue'.format(x) for x in revenue_list]
expenditures_perc_list = ['{}_perc_expenditures'.format(x) for x in expenditures_list]    

df[revenue_perc_list] = df[revenue_list].div(df['4700'], axis=0)
df[expenditures_perc_list] = df[expenditures_list].div(df['5100'], axis=0)

In [ ]:
#replace infinite values with nan
df = df.replace([np.inf, -np.inf], np.nan)

In [ ]:
#save output for exploratory analysis
df.to_csv('E:\Projects\canadacharities\data\interim\charity_data.csv', index=False)

# Preview Data

In [23]:
display(df)

,BN/Registration Number,Designation Code,Category Code,Sub-Category Code_x,Legal Name_x,Account Name,Registration Date,Mailing Address,City,Province,Postal Code,Country,Contact Phone,Contact Email,Contact Url,Bn/Registration Number_X,Fiscal Period End_X,Form Id_X,1200 Program Area Code,1200 Percent,1210 Program Area Code,1210 Percent,1220 Program Area Code,1220 Percent,1510 Subordinate Position To A Parent Organization?,1510 Parent Business Number,1510 Parent Name,1570,1600,1800,2000,2100,2400,5030_X,5031,5032,2500,2510,2530,2540,2550,2560,2570,2575,2580,2590,2600,2610,2620,2630,2640,2650,2660,2700,5450,5460,2730,2740,2750,2760,2770,2780,2790,2800,3200,3400,3900,4000,5800,5810,5820,5830,Bn/Registration Number_Y,Fiscal Period End_Y,Form Id_Y,Financial Indicator (D Or 6),5030 Indicator (C Or 6),4020,4050,4100,4110,4120,4130,4140,4150,4155,4160,4165,4166,4170,4180,4200,4250,4300,4310,4320,4330,4350,4400,4490,4500,5610,4505,4510,4530,4540,4550,4560,4565,4570,4571,4575,4580,4590,4600,4610,4620,4630,4640,4650,4655,4700,4800,4810,4820,4830,4840,4850,4860,4870,4880,4890,4891,4900,4910,4920,4930,4950,5000,5010,5020,5030_Y,5040,5050,5100,5500,5510,5750,5900,5910,Category,Sub-Category Code_y,Sub-Category,Designation,Program Code_x,1200 Program Area,Program Code_y,1210 Program Area,Program Code,1220 Program Area,Legal Name_y,Revoked Date,Revoked Reason
0,100021237RR0001,C,10,99,ACADEMIE LAFONTAINE INC.,ACADEMIE LAFONTAINE INC.,1987-7-1 00:00:00,"2171, BOUL. MAURICE",ST-JÉRÔME,QC,J7Y4M7,CA,(450)431-3733,ALARIEH@ACADEMIELAFONTAINE.QC.CA,WWW.ACADEMIELAFONTAINE.QC.CA,100021237RR0001,2018-06-30 00:00,23,C5,100,NaN,NaN,NaN,NaN,N,NaN,NaN,N,N,N,N,N,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,Y,N,N,N,N,N,N,100021237RR0001,2018-6-30 00:00:00,23,6,NaN,A,NaN,$4827488.00,NaN,$324998.00,NaN,NaN,NaN,$23113453.00,$908602.00,NaN,NaN,$19194.00,NaN,$29193735.00,NaN,$10212432.00,$56522.00,NaN,$13397700.00,$23666654.00,NaN,NaN,$435688.00,NaN,NaN,NaN,NaN,NaN,$9871247.00,NaN,NaN,NaN,NaN,NaN,$56682.00,NaN,NaN,$196770.00,$7290865.00,NaN,$749774.00,$3289138.00,NaN,$21890164.00,$455284.00,$30347.00,$964557.00,NaN,$1103122.00,$857216.00,$2343047.00,$132426.00,$13809052.00,NaN,NaN,$1324005.00,NaN,$12655.00,NaN,$21031711.00,$11511820.00,$3055036.00,NaN,NaN,$6464855.00,NaN,$21031711.00,NaN,NaN,NaN,NaN,NaN,Teaching Institutions,1,Preschool (early-childhood education / junior ...,Charitable Organization,C5,Independent schools and boards,NaN,NaN,NaN,NaN,NaN,NaT,NaN
1,100021237RR0001,C,10,99,ACADEMIE LAFONTAINE INC.,ACADEMIE LAFONTAINE INC.,1987-7-1 00:00:00,"2171, BOUL. MAURICE",ST-JÉRÔME,QC,J7Y4M7,CA,(450)431-3733,ALARIEH@ACADEMIELAFONTAINE.QC.CA,WWW.ACADEMIELAFONTAINE.QC.CA,100021237RR0001,2018-06-30 00:00,23,C5,100,NaN,NaN,NaN,NaN,N,NaN,NaN,N,N,N,N,N,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,Y,N,N,N,N,N,N,100021237RR0001,2018-6-30 00:00:00,23,6,NaN,A,NaN,$4827488.00,NaN,$324998.00,NaN,NaN,NaN,$23113453.00,$908602.00,NaN,NaN,$19194.00,NaN,$29193735.00,NaN,$10212432.00,$56522.00,NaN,$13397700.00,$23666654.00,NaN,NaN,$435688.00,NaN,NaN,NaN,NaN,NaN,$9871247.00,NaN,NaN,NaN,NaN,NaN,$56682.00,NaN,NaN,$196770.00,$7290865.00,NaN,$749774.00,$3289138.00,NaN,$21890164.00,$455284.00,$30347.00,$964557.00,NaN,$1103122.00,$857216.00,$2343047.00,$132426.00,$13809052.00,NaN,NaN,$1324005.00,NaN,$12655.00,NaN,$21031711.00,$11511820.00,$3055036.00,NaN,NaN,$6464855.00,NaN,$21031711.00,NaN,NaN,NaN,NaN,NaN,Teaching Institutions,2,Kindergarten,Charitable Organization,C5,Independent schools and boards,NaN,NaN,NaN,NaN,NaN,NaT,NaN
2,100021237RR0001,C,10,99,ACADEMIE LAFONTAINE INC.,ACADEMIE LAFONTAINE INC.,1987-7-1 00:00:00,"2171, BOUL. MAURICE",ST-JÉRÔME,QC,J7Y4M7,CA,(450)431-3733,ALARIEH@ACADEMIELAFONTAINE.QC.CA,WWW.ACADEMIELAFONTAINE.QC.CA,100021237RR0001,2018-06-30 00:00,23,C5,100,NaN,NaN,NaN,NaN,N,NaN,NaN,N,N,N,N,N,N,NaN,NaN,NaN,NaN,